<a href="https://colab.research.google.com/github/Deltan2002/RNN/blob/main/cypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip crypto_data.zip

Archive:  crypto_data.zip
replace crypto_data/BCH-USD.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
from sklearn import preprocessing
from collections import deque
import numpy as np
import random
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM,BatchNormalization
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint


# df = pd.read_csv('crypto_data/LTC-USD.csv',names=['time','low','high','open','close','volume'])
# print(df.head())

SEQ_LEN=60
FUTURE_PERIOD_PREDICT=3
RATIO_TO_PREDICT='LTC-USD'
EPOCHS=2
BATCH_SIZE=64
NAME=f'{RATIO_TO_PREDICT}-{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}'



def classify(current,future):
  if float(future) > float(current):
    return 1
  else:
    return 0


main_df = pd.DataFrame()
ratios = ['BTC-USD','LTC-USD','ETH-USD','BCH-USD']


In [ ]:
def preprocess_df(df):
  df.drop(['future'], axis=1, inplace=True)

  for col in df.columns:
    if col!='target':
      df[col] = df[col].pct_change()
      df.dropna(inplace=True)
      df[col] = preprocessing.scale(df[col].values) # data b/w 0 ad 1

  df.dropna(inplace=True)

  sequential_data=[]
  prev_days = deque(maxlen=SEQ_LEN)

  if df is not None:
    for i in df.values:
      prev_days.append([n for n in i[:-1]])
      if len(prev_days) == SEQ_LEN:
        sequential_data.append([np.array(prev_days), i[-1]])
  random.shuffle(sequential_data)


  buys=[]
  sells=[]

  for seq,target in sequential_data:
    if target ==0:
      sells.append([seq,target])
    else:
      buys.append([seq,target])

  random.shuffle(buys)
  random.shuffle(sells)

  lower =  min(len(buys),len(sells))

  buys = buys[:lower]
  sells = sells[:lower]


  sequential_data = buys+sells
  random.shuffle(sequential_data)


  X =[]
  y = []

  for seq,target in sequential_data:
    X.append(seq)
    y.append(target)
  return np.array(X),y



In [ ]:
for ratio in ratios:
  dataset = f'crypto_data/{ratio}.csv'

  df = pd.read_csv(dataset,names=['time','low','high','open','close','volume'])
  df.rename(columns={'close':f'{ratio}_close','volume': f'{ratio}_volume'},inplace=True)

  df.set_index('time',inplace=True)
  df = df[[f'{ratio}_close',f'{ratio}_volume']]
  # print(df.head())



  if len(main_df) ==0:
    main_df =df
  else:
    main_df = main_df.join(df,how='outer' ,rsuffix=f'_{ratio}')


main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))


main_df.fillna(method="ffill", inplace=True)
main_df.dropna(inplace=True)



print(main_df[[f'{RATIO_TO_PREDICT}_close','future','target']].head(10))


            LTC-USD_close     future  target
time                                        
1528968720      96.660004  96.389999       0
1528968780      96.570000  96.519997       0
1528968840      96.500000  96.440002       0
1528968900      96.389999  96.470001       1
1528968960      96.519997  96.400002       0
1528969020      96.440002  96.400002       0
1528969080      96.470001  96.400002       0
1528969140      96.400002  96.400002       0
1528969200      96.400002  96.400002       0
1528969260      96.400002  96.449997       1


In [ ]:
times = sorted(main_df.index.values)
last_5pct = times[-int(0.05*len(times))]

validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]



In [ ]:
train_x,train_y = preprocess_df(main_df)
validation_x,validation_y = preprocess_df(validation_main_df)


train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
validation_x = np.asarray(validation_x)
validation_y = np.asarray(validation_y)

In [ ]:
print(f'Train Data: {len(train_x)} Validation: {len(validation_x)}')
print(f'Dont Buys: {np.sum(train_y == 0)}, Buys: {np.sum(train_y == 1)}')
print(f'VALIDATION Dont Buys: {np.sum(validation_y == 0)}, Buys: {np.sum(validation_y == 1)}')

Train Data: 80770 Validation: 3848
Dont Buys: 40385, Buys: 40385
VALIDATION Dont Buys: 1924, Buys: 1924


In [ ]:
from keras.src.optimizers import optimizer
model = Sequential()
model.add(LSTM(128,input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())


model.add(LSTM(128,input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())


model.add(LSTM(128,input_shape=(train_x.shape[1:])))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))


model.add(Dense(2, activation='softmax'))

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)

opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])




tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
filepath = 'RNN_Final-{epoch:02d}-{val_accuracy:.3f}'
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'))

history = model.fit(
    train_x, train_y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_x, validation_y),
    callbacks=[tensorboard, checkpoint],
)

# Score model
score = model.evaluate(validation_x, validation_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save model
# model.save("models/{}".format(NAME))



Epoch 1/2
1263/1263 [==============================] - 345s 269ms/step - loss: 0.6966 - accuracy: 0.5017 - val_loss: 0.6932 - val_accuracy: 0.5003
Epoch 2/2
1263/1263 [==============================] - 343s 271ms/step - loss: 0.6936 - accuracy: 0.5035 - val_loss: 0.6932 - val_accuracy: 0.5000
Test loss: 0.6931525468826294
Test accuracy: 0.5
